# Learning and Decision Making

## Laboratory 3: Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the predator-prey domain described in the Homework and which you described as a Markov decision process.

<img src="toroidal-world.png" width="400px">

Recall that:

* toroidal world "wraps around", i.e., an individual exiting through any of the four sides of the grid reenters on the opposite side (see figure above).

* At each time step, the hare selects uniformly at random one of the four directions (up, down, left, and right) and moves to the adjacent cell in that direction with a probability 0.4. With a probability 0.6 it remains in the same cell. 

* The wolf, on the other hand, can select at each time step one of five actions---up (_U_), down (_D_), left (_L_) and right (_R_) or stay (_S_). If it selects action _S_, it remains in the same cell with probability 1.0. Otherwise, the other 4 actions succeed in moving the wolf to the adjacent cell in the corresponding direction with a probability 0.8 and fail with a probability 0.2. 

* The goal of the wolf is to catch the hare.

---

#### Activity 1.        

Implement your Markov decision process in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* For each action, define a `numpy` array with the corresponding transition probabilities;
* Define a `numpy`array with the costs

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note 1**: Don't forget to import `numpy`.

**Note 2**: You can define the transition probability matrices for each of the two individuals and then build the combined transition probability matrices using the `numpy.kron` function.

---

In [34]:
import numpy as np

X = [(1,1), (1,2), (1,3), (1,4), (2,1), (2,2), (2,3), (2,4), (3,1), (3,2), (3,3), (3,4), (4,1), (4,2), (4,3), (4,4)]

A = ['S', 'U', 'D', 'L', 'R']

P_rabbit = np.array([[0.6, 0.2, 0.2, 0],[0.2, 0.6, 0, 0.2], [0.2, 0, 0.6, 0.2], [0,0.2,0.2,0.6]])

P_wolf_stay = np.array([[1,0,0,0],[0,1,0,0], [0,0,1,0], [0,0,0,1]])

P_wolf_up = np.array([[0.2, 0, 0.8, 0],[0, 0.2, 0, 0.8], [0.8, 0, 0.2, 0], [0,0.8, 0, 0.2]])

P_wolf_down = P_wolf_up

P_wolf_left = np.array([[0.2, 0.8, 0, 0], [0.8, 0.2, 0, 0], [0,0,0.2, 0.8], [0,0,0.8, 0.2]])

P_wolf_right = P_wolf_left

P_stay = np.kron(P_wolf_stay, P_rabbit)

P_up = np.kron(P_wolf_up, P_rabbit)

P_down = np.kron(P_wolf_down, P_rabbit)

P_left = np.kron(P_wolf_left, P_rabbit)

P_right = np.kron(P_wolf_right, P_rabbit)

# Cost Logic: the wolf is penalized for every step that the prey is “free”
C = np.array([[0,0,0,0,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1], [1,1,1,1,1], [0,0,0,0,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1], [0,0,0,0,0], [1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1], [0,0,0,0,0]])


### 2. Prediction

You are now going to evaluate a given policy, computing the corresponding cost-to-go.

---

#### Activity 2.

Describe the policy that, in each state $(w, h)$, always moves the wolf to the cell closest to the hare. If multiple such cells exist, the wolf should select randomly between the two.

For example, suppose that the wolf is in cell 1 and the hare in cell 4 (figure above, left). The wolf should then select randomly between the actions _U_, _D_ (which move the wolf to cell 3), _L_ and _R_ (which move the wolf to cell 2). Conversely, if the wolf is in cell 1 and the hare in cell 3 (figure above, right), the wolf should select randomly between the two actions _U_ and _D_ (which move the wolf to cell 3).

**Note:** The policy should be described as a vector with as many rows as there are states and as many columns as there are actions, where the entry _xa_ has the probability of selecting action _a_ in state _x_.

---

In [35]:
ups_downs = {1:3, 3:1, 2:4, 4:2}
lefts_rights = {1:2, 2:1, 3:4, 4:3}
diagonals = {1:4, 4:1, 3:2, 2:3}

#Function that given a state, returns the probability vector of choosing each action according to the described policy
def policy_probability(state):
    if state[0] == state[1]: #If they're on the same patch, we stay
        return [1,0,0,0,0]
    elif state[1] == diagonals[state[0]]: #If the rabbit is on the diagonal, we can choose either Up, Down, Left or Right
        return [0,1/4, 1/4, 1/4, 1/4]
    elif state[1] == ups_downs[state[0]]: #If the rabbit is above or below, we can choose either Up or Down
        return [0,1/2, 1/2, 0, 0]
    else: #If the rabbit is left or right, we can choose either Left or Right
        return [0,0, 0, 1/2, 1/2]

policy = []
for state in X:
    policy += [policy_probability(state)]

policy = np.array(policy)
print(policy)


[[ 1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 1.    0.    0.    0.    0.  ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 1.    0.    0.    0.    0.  ]]


---

#### Activity 3.

Compute the cost-to-go function $J^\pi$ associated with the policy from Activity 2. Use $\gamma=0.99$.

---

In [36]:
#Dictionary that maps the action's index with its probability matrix
action_matrixes = {0:P_stay, 1:P_up, 2:P_down, 3:P_left, 4:P_right}

gamma = 0.99

#Expected cost at each state
def cost_policy(state): 
    stateIndex = X.index(state)
    soma = 0
    for action in range(len(A)):
        soma = soma + policy[stateIndex][action] * C[stateIndex][action]
    return soma

#Transition probabiliy
def probabilities_policy(currentState, nextState):
    soma = 0
    currentStateIndex = X.index(currentState)
    nextStateIndex = X.index(nextState)
    for action in range(len(A)):
        soma = soma + policy[currentStateIndex][action] * action_matrixes[action][currentStateIndex][nextStateIndex]
    return soma

#Transition probabilities in matrix form
probabilities_policy_matrix = []
for currentState in X:
    row = []
    for nextState in X:
        row += [probabilities_policy(currentState, nextState)]
    probabilities_policy_matrix += [row]
probabilities_policy_matrix = np.array(probabilities_policy_matrix)

#Costs for each state in matrix form
cost_policy_matrix = []
for state in X:
    cost_policy_matrix += [cost_policy(state)]
cost_policy_matrix = np.array(cost_policy_matrix)

J_policy = np.dot(np.linalg.inv((np.eye(len(X)) - gamma * probabilities_policy_matrix)), cost_policy_matrix)
print(J_policy)

[ 43.88841893  44.99671233  44.99671233  45.51480752  44.99671233
  43.88841893  45.51480752  44.99671233  44.99671233  45.51480752
  43.88841893  44.99671233  45.51480752  44.99671233  44.99671233
  43.88841893]


### 3. Control

In this section you are going to compare value and policy iteration, both in terms of time and number of iterations.

---

#### Activity 4

Show that the policy in Activity 3 is optimal: use value iteration to compute $J^*$ and show that $J^*=J^\pi$. Track the time and the number of iterations taken to compute $J^*$.

**Note 1:** Stop the algorithm when the error between iterations is smaller than $10^{-8}$.

**Note 2:** You may find useful the function ``time()`` from the module ``time``.

---

In [37]:
import time
J = np.zeros((len(X),1))
err = 1
i = 0
start_time = time.time()
while err > 1e-8:
    Q_stay = C[:,0:1] + gamma * P_stay.dot(J)
    Q_up = C[:,1:2] + gamma * P_up.dot(J)
    Q_down = C[:,2:3] + gamma * P_down.dot(J)
    Q_left = C[:,3:4] + gamma * P_left.dot(J)
    Q_right = C[:,4:5] + gamma * P_right.dot(J)
    J_new = np.min([Q_up, Q_down,Q_left,Q_right,Q_stay], axis=0)
    err = np.linalg.norm(J_new-J)
    i += 1
    J = J_new
finish_time = time.time() - start_time  
print("Finished with " + str(i)+" iterations and in " + str(finish_time) +" seconds")
print(J) #As J_policy == J, then our policy is optimal!

Finished with 1891 iterations and in 0.08011555671691895 seconds
[[ 43.88841868]
 [ 44.99671209]
 [ 44.99671209]
 [ 45.51480727]
 [ 44.99671209]
 [ 43.88841868]
 [ 45.51480727]
 [ 44.99671209]
 [ 44.99671209]
 [ 45.51480727]
 [ 43.88841868]
 [ 44.99671209]
 [ 45.51480727]
 [ 44.99671209]
 [ 44.99671209]
 [ 43.88841868]]


---

#### Activity 5

Compute once again the optimal policy now using policy iteration. Track the time and number of iterations taken and compare to those of Activity 4.

**Note:** If you find that numerical errors affect your computations (especially when comparing two values/arrays) you may use the `numpy` function `isclose` with adequately set absolute and relative tolerance parameters (e.g., $10^{-8}$).

---

In [38]:
i = 0
pi = policy #Our initial policy
quit = False
start_time = time.time()
while not quit:
    cpi = np.diag(pi[:,0]).dot(C[:,0]) + np.diag(pi[:,1]).dot(C[:,1]) + np.diag(pi[:,2]).dot(C[:,2]) + np.diag(pi[:,3]).dot(C[:,3]) + np.diag(pi[:,4]).dot(C[:,4])
    Ppi = np.diag(pi[:,0]).dot(P_stay) + np.diag(pi[:,1]).dot(P_up) + np.diag(pi[:,2]).dot(P_down) + np.diag(pi[:,3]).dot(P_left) + np.diag(pi[:,4]).dot(P_right)
    J = np.linalg.inv(np.eye(16) - gamma * Ppi).dot(cpi)
    J_fix = []
    for el in J:
        J_fix += [[el]]
    J = np.array(J_fix)
    Q_stay = C[:,0:1] + gamma * P_stay.dot(J)
    Q_up = C[:,1:2] + gamma * P_up.dot(J)
    Q_down = C[:,2:3] + gamma * P_down.dot(J)
    Q_left = C[:,3:4] + gamma * P_left.dot(J)
    Q_right = C[:,4:5] + gamma * P_right.dot(J)
    pinew = np.zeros((16,5))
    pinew[:, 0, None] = np.isclose(Q_stay, np.min([Q_stay,Q_up, Q_down, Q_left, Q_right], axis=0), atol=1e-8, rtol=1e-8).astype(int)
    pinew[:, 1, None] = np.isclose(Q_up, np.min([Q_stay,Q_up, Q_down, Q_left, Q_right], axis=0), atol=1e-8, rtol=1e-8).astype(int)
    pinew[:, 2, None] = np.isclose(Q_down, np.min([Q_stay,Q_up, Q_down, Q_left, Q_right], axis=0), atol=1e-8, rtol=1e-8).astype(int)
    pinew[:, 3, None] = np.isclose(Q_left, np.min([Q_stay,Q_up, Q_down, Q_left, Q_right], axis=0), atol=1e-8, rtol=1e-8).astype(int)
    pinew[:, 4, None] = np.isclose(Q_right, np.min([Q_stay,Q_up, Q_down, Q_left, Q_right], axis=0), atol=1e-8, rtol=1e-8).astype(int)
    pinew = pinew / np.sum(pinew, axis=1, keepdims=True)
    quit = (pi == pinew).all()
    pi = pinew
    i += 1
finish_time = time.time() - start_time  
print("Finished with " + str(i)+" iterations and in " + str(finish_time) +" seconds")
print(pi)

Finished with 1 iterations and in 0.0009996891021728516 seconds
[[ 1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 1.    0.    0.    0.    0.  ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 1.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 0.    0.25  0.25  0.25  0.25]
 [ 0.    0.5   0.5   0.    0.  ]
 [ 0.    0.    0.    0.5   0.5 ]
 [ 1.    0.    0.    0.    0.  ]]


### 4. Simulation

Finally, in this section you will check whether the theoretical computations of the cost-to-go actually correspond to the cost incurred by an agent following a policy.

---

#### Activity 6

Starting in each of the two states $x$ in the initial figure, 

* Generate **100** trajectories of 10,000 steps each, following the optimal policy for the MDP. 
* For each trajectory, compute the accumulated (discounted) cost. 
* Compute the average cost over the 100 trajectories.
* Compare the resulting value with that computed in Activity 4 for the two states. 

** Note:** The simulation may take a bit of time, don't despair ☺️.

---

In [40]:
for initial_state in [(1,4),(1,3)]:
    for trajectory in range(100):
        soma = 0
        for step in range(10000):
            soma = soma + gamma**step * cost_policy(state)
            state = X[np.random.choice(range(len(X)), p=probabilities_policy_matrix[X.index(state)])]
    print("Starting at " + str(initial_state)+" we get a discounted cost of " + str(soma) + " which is a " + str(abs(soma - J[X.index(initial_state)][0])) + " difference from the theoretical value")

Starting at (1, 4) we get a discounted cost of 43.3461972497 which is a 2.16861027122 difference from the theoretical value
Starting at (1, 3) we get a discounted cost of 40.8721036945 which is a 4.12460863979 difference from the theoretical value
